In [2]:
import pandas as pd
from lsst.rsp import get_tap_service

service = get_tap_service("ssotap")

query_ssObjectIds = "SELECT DISTINCT sso.ssObjectId FROM dp03_catalogs_10yr.SSObject AS sso"

ssObjectIds = service.search(query_ssObjectIds)
ssObjectIds

<DALResultsTable length=4443479>
     ssObjectId     
       int64        
--------------------
-9223370383071521539
-9223366535898786459
-9223365099512229794
-9223364443532342230
-9223361410996246632
-9223354132337021199
-9223352335202182781
-9223351106075435288
-9223343361837155588
                 ...
 9223333118972405713
 9223337017087558055
 9223345276844401162
 9223346423002359784
 9223352613567217422
 9223353272167950944
 9223364740177399476
 9223365960393921544
 9223366833108713925
 9223370430250665087

In [3]:
data = {'ssObjectIds': ssObjectIds}
df = pd.DataFrame(data)
df.to_csv('ssObjectIds.csv')

In [42]:
num_ids = len(df)
iter_size = 50000
num_iter = 100 #int(num_ids/iter_size) + 1
print("Number of object IDs: "+str(num_ids))
print("Number of iterations: "+str(num_iter))

Number of object IDs: 4443479
Number of iterations: 10


In [43]:
start_idx = 3998995
df_results = pd.DataFrame()

for i in range(num_iter):
    end_idx = start_idx+iter_size
    if end_idx > num_ids:
        id_list = "(" + ", ".join(str(value1) for value1 in ssObjectIds['ssObjectId'][start_idx:-1]) + ")"
        query1 = """SELECT sso.ssObjectId, MIN(sss.diaSourceId) AS min_diaSourceId
                    FROM dp03_catalogs_10yr.SSObject AS sso
                    JOIN dp03_catalogs_10yr.SSSource AS sss ON sso.ssObjectId = sss.ssObjectId
                    WHERE sso.ssObjectId IN {}
                    GROUP BY sso.ssObjectId
                    ORDER BY sso.ssObjectId ASC""".format(id_list)
   
        results1 = service.search(query1)

        string_list = "(" + ", ".join(str(value) for value in results1['min_diaSourceId']) + ")"

        query2 = """SELECT mpc.ssObjectId, sss.diaSourceId AS min_diaSourceId,
                    mpc.fullDesignation, mpc.incl, mpc.e, mpc.q, mpc.node, 
                    mpc.peri, mpc.mpcH, sso.g_H, sso.r_H, sso.i_H, sso.z_H, sso.u_H, sso.y_H, 
                    sss.heliocentricDist, sss.heliocentricVX, sss.heliocentricVY, sss.heliocentricVZ
                    FROM dp03_catalogs_10yr.MPCORB AS mpc
                    JOIN dp03_catalogs_10yr.SSObject AS sso ON mpc.ssObjectId = sso.ssObjectId
                    JOIN dp03_catalogs_10yr.SSSource AS sss ON mpc.ssObjectId = sss.ssObjectId
                    WHERE sss.diaSourceId IN {}
                    ORDER BY mpc.ssObjectId ASC""".format(string_list)

        results2 = service.search(query2)
        results2_astropy = results2.to_table()
        df_results2 = results2_astropy.to_pandas()
        df_results = pd.concat([df_results, df_results2], ignore_index=True)
        start_idx = end_idx
        print("Reached the end")
        break
    
    id_list = "(" + ", ".join(str(value1) for value1 in ssObjectIds['ssObjectId'][start_idx:end_idx]) + ")"
    query1 = """SELECT sso.ssObjectId, MIN(sss.diaSourceId) AS min_diaSourceId
                FROM dp03_catalogs_10yr.SSObject AS sso
                JOIN dp03_catalogs_10yr.SSSource AS sss ON sso.ssObjectId = sss.ssObjectId
                WHERE sso.ssObjectId IN {}
                GROUP BY sso.ssObjectId
                ORDER BY sso.ssObjectId ASC""".format(id_list)
   
    results1 = service.search(query1)

    string_list = "(" + ", ".join(str(value) for value in results1['min_diaSourceId']) + ")"

    query2 = """SELECT mpc.ssObjectId, sss.diaSourceId AS min_diaSourceId,
                mpc.fullDesignation, mpc.incl, mpc.e, mpc.q, mpc.node, 
                mpc.peri, mpc.mpcH, sso.g_H, sso.r_H, sso.i_H, sso.z_H, sso.u_H, sso.y_H, 
                sss.heliocentricDist, sss.heliocentricVX, sss.heliocentricVY, sss.heliocentricVZ
                FROM dp03_catalogs_10yr.MPCORB AS mpc
                JOIN dp03_catalogs_10yr.SSObject AS sso ON mpc.ssObjectId = sso.ssObjectId
                JOIN dp03_catalogs_10yr.SSSource AS sss ON mpc.ssObjectId = sss.ssObjectId
                WHERE sss.diaSourceId IN {}
                ORDER BY mpc.ssObjectId ASC""".format(string_list)
    
    results2 = service.search(query2)
    results2_astropy = results2.to_table()
    df_results2 = results2_astropy.to_pandas()
    df_results = pd.concat([df_results, df_results2], ignore_index=True)
    
    start_idx = end_idx
    print("just finished the {}th iteration".format(i))

print(df_results)
df_results.to_csv('All_Objects_3998995-4.4M.csv')

just finished the 0th iteration
just finished the 1th iteration
just finished the 2th iteration
just finished the 3th iteration
just finished the 4th iteration
just finished the 5th iteration
just finished the 6th iteration
just finished the 7th iteration
Reached the end
                 ssObjectId      min_diaSourceId fullDesignation       incl  \
0       7379422925054239185 -9215685903882478929  2011 S100CXiSa   8.884670   
1       7379423487943011983 -8679690379402301655  2011 S0000s0na   7.989721   
2       7379423914542945358 -8902464937402071432  2011 S100TxA9a  14.962840   
3       7379424494384829620 -9086920484863484594   2011 2009 KC4   7.396470   
4       7379425521487385428 -9196990815613587783  2011 S1008mgka   7.725190   
...                     ...                  ...             ...        ...   
444358  9223352613567217422 -9056497399985485806  2011 2002 VP26   2.125300   
444359  9223353272167950944 -9203810900853160883  2011 S101y1kEa  14.906890   
444360  922336474